In [2]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

Clave_Acceso = "82f72b70efe3582c843f7c2ea378986b" 
valor_faltante = "Sin dato"

def convertir_dato(value):
    
    # Convierte a entero o retorna valor_faltante si no es un número válido.
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return valor_faltante

def obtener_heroe(hero_id):
    
    #Obtiene la información de un héroe por ID.
    
    url = f"https://superheroapi.com/api/{Clave_Acceso}/{hero_id}"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException:
        return None  # Si hay error, devuelve None

    if data.get("response") != "success":
        return None
        
    #Extraemos la información que nececitamos y armamos un diccionario limpio.
    
    stats = data.get("powerstats", {})
    appearance = data.get("appearance", {})
    biography = data.get("biography", {})

    hero = {
        "ID": data.get("id", valor_faltante),
        "Nombre": data.get("name", valor_faltante),
        "Inteligencia": convertir_dato(stats.get("intelligence", valor_faltante)),
        "Fuerza": convertir_dato(stats.get("strength", valor_faltante)),
        "Velocidad": convertir_dato(stats.get("speed", valor_faltante)),
        "Durabilidad": convertir_dato(stats.get("durability", valor_faltante)),
        "Potencia": convertir_dato(stats.get("power", valor_faltante)),
        "Combate": convertir_dato(stats.get("combat", valor_faltante)),
        "Genero": appearance.get("gender", valor_faltante),
        "Etnia": appearance.get("race", valor_faltante),
        "Altura": appearance.get("height", ["Sin dato", "Sin dato"])[1] if len(appearance.get("height", [])) > 1 else valor_faltante, 
        "Peso": appearance.get("weight", ["Sin dato", "Sin dato"])[1] if len(appearance.get("weight", [])) > 1 else valor_faltante,
        "Color_Ojos": appearance.get("eye-color", valor_faltante),
        "Color_Pelo": appearance.get("hair-color", valor_faltante),
        "Nombre Completo": biography.get("full-name", valor_faltante),
        "Alias": ", ".join(biography.get("aliases", [])) if biography.get("aliases") else valor_faltante,
        "Lugar de nacimiento": biography.get("place-of-birth", valor_faltante),
        "Primera aparición": biography.get("first-appearance", valor_faltante),
        "Editor": biography.get("publisher", valor_faltante),
        "Alineación": biography.get("alignment", valor_faltante)
    }
     #datos como altura y peso usamos esa funcion para extraer la altura y el peso en unidades métricas de la API ,
     #si existe el dato real nos lo refleja (cm / kg) y sino nos da "Sin Dato" , y usamos la segunda posicion[1], para que sea en cm y kg.

    return hero

def obtener_heroes_completo_paralelo(max_id=731, guardar_csv=True, max_workers=20):
    heroes_data = []
    #creamos un grupo de threads para extraer varias lineas a la vez, y asi extraer los datos mas rapido, 20 heroes por cada peticion.
    #mandamos tareas y las procesamos mientras se completen y con tqdm, creamos una barra de progreso y mientras cada future termina , aumenta .
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(obtener_heroe, hero_id): hero_id for hero_id in range(1, max_id + 1)}

        for future in tqdm(as_completed(futures), total=max_id, desc="Descargando héroes"):
            hero = future.result()
            if hero:
                heroes_data.append(hero)

    df_heroes = pd.DataFrame(heroes_data)

    if guardar_csv:
        df_heroes.to_csv("heroes_data.csv", index=False)
        print("Datos guardados en 'heroes_data.csv'")

    return df_heroes

df = obtener_heroes_completo_paralelo()
df.head()



Descargando héroes: 100%|██████████| 731/731 [00:38<00:00, 18.78it/s]

Datos guardados en 'heroes_data.csv'


,ID,Nombre,Inteligencia,Fuerza,Velocidad,Durabilidad,Potencia,Combate,Genero,Etnia,Altura,Peso,Color_Ojos,Color_Pelo,Nombre Completo,Alias,Lugar de nacimiento,Primera aparición,Editor,Alineación
0,6,Absorbing Man,38,80,25,100,98,64,Male,Human,193 cm,122 kg,Blue,No Hair,Carl Creel,"Greithoth, ""Crusher"" Creel, Rocky Davis, Dynam...","New York City, New York","Daredevil #1 (April, 1964) (As Rocky Davis)",Marvel Comics,bad
1,20,Amazo,63,100,83,100,100,100,Male,Android,257 cm,173 kg,Red,-,,"Professor Ivos Amazing Android, Timazo, Humazo...",-,"Brave and the Bold #30 (July, 1960)",DC Comics,bad
2,10,Agent Bob,10,8,13,5,5,20,Male,Human,178 cm,81 kg,Brown,Brown,Bob,"Bob, agent of Hydra, Bob, agent of A.I.M",-,"Cable & Deadpool #38 (May, 2007)",Marvel Comics,good
3,3,Abin Sur,50,90,53,64,99,65,Male,Ungaran,185 cm,90 kg,Blue,No Hair,,Lagzia,Ungara,"Showcase #22 (October, 1959)",DC Comics,good
4,12,Air-Walker,50,85,100,85,100,40,Male,null,188 cm,108 kg,Blue,White,Gabriel Lan,-,"Xandar, a planet in the Tranta system, Androme...",Fantastic Four #120,Marvel Comics,bad


<h2>BATMAN🦇<h2>

In [6]:
obtener_heroe(70)

{'ID': '70',
 'Nombre': 'Batman',
 'Inteligencia': 100,
 'Fuerza': 26,
 'Velocidad': 27,
 'Durabilidad': 50,
 'Potencia': 47,
 'Combate': 100,
 'Genero': 'Male',
 'Etnia': 'Human',
 'Altura': '188 cm',
 'Peso': '95 kg',
 'Color_Ojos': 'blue',
 'Color_Pelo': 'black',
 'Nombre Completo': 'Bruce Wayne',
 'Alias': 'Insider, Matches Malone',
 'Lugar de nacimiento': 'Crest Hill, Bristol Township; Gotham County',
 'Primera aparición': 'Detective Comics #27',
 'Editor': 'DC Comics',
 'Alineación': 'good'}

In [16]:
heroe_70 = obtener_heroe(70)
df_70 = pd.DataFrame([heroe_70])
df_70

,ID,Nombre,Inteligencia,Fuerza,Velocidad,Durabilidad,Potencia,Combate,Genero,Etnia,Altura,Peso,Color_Ojos,Color_Pelo,Nombre Completo,Alias,Lugar de nacimiento,Primera aparición,Editor,Alineación
0,70,Batman,100,26,27,50,47,100,Male,Human,188 cm,95 kg,blue,black,Bruce Wayne,"Insider, Matches Malone","Crest Hill, Bristol Township; Gotham County",Detective Comics #27,DC Comics,good


<h2>DeadPool😡<h2>

In [8]:
obtener_heroe(213)

{'ID': '213',
 'Nombre': 'Deadpool',
 'Inteligencia': 69,
 'Fuerza': 32,
 'Velocidad': 50,
 'Durabilidad': 100,
 'Potencia': 100,
 'Combate': 100,
 'Genero': 'Male',
 'Etnia': 'Mutant',
 'Altura': '188 cm',
 'Peso': '95 kg',
 'Color_Ojos': 'Brown',
 'Color_Pelo': 'No Hair',
 'Nombre Completo': 'Wade Wilson',
 'Alias': 'Wade Wilson, Jack, Chiyonosake ("the Wolf of the Rice Wine"), Rhodes, Corpus, Lopez, Hobgoblin, Thom Cruz, Peter Parker',
 'Lugar de nacimiento': 'Canada',
 'Primera aparición': 'New Mutants #98 (February, 1991)',
 'Editor': 'Evil Deadpool',
 'Alineación': 'neutral'}

In [17]:
heroe_213 = obtener_heroe(213)
df_213 = pd.DataFrame([heroe_213])
df_213

,ID,Nombre,Inteligencia,Fuerza,Velocidad,Durabilidad,Potencia,Combate,Genero,Etnia,Altura,Peso,Color_Ojos,Color_Pelo,Nombre Completo,Alias,Lugar de nacimiento,Primera aparición,Editor,Alineación
0,213,Deadpool,69,32,50,100,100,100,Male,Mutant,188 cm,95 kg,Brown,No Hair,Wade Wilson,"Wade Wilson, Jack, Chiyonosake (""the Wolf of t...",Canada,"New Mutants #98 (February, 1991)",Evil Deadpool,neutral


<h2>Spider-Man🕷️🕸️<h2>

In [9]:
obtener_heroe(620)

{'ID': '620',
 'Nombre': 'Spider-Man',
 'Inteligencia': 90,
 'Fuerza': 55,
 'Velocidad': 67,
 'Durabilidad': 75,
 'Potencia': 74,
 'Combate': 85,
 'Genero': 'Male',
 'Etnia': 'Human',
 'Altura': '178 cm',
 'Peso': '74 kg',
 'Color_Ojos': 'Hazel',
 'Color_Pelo': 'Brown',
 'Nombre Completo': 'Peter Parker',
 'Alias': 'Spiderman, Bag-Man, Black Marvel, Captain Universe, Dusk, Green Hood, Hornet, Mad Dog 336, Peter Palmer, Prodigy, Ricochet, Scarlet Spider, Spider-Boy, Spider-Hulk, Spider-Morphosis',
 'Lugar de nacimiento': 'New York, New York',
 'Primera aparición': 'Amazing Fantasy #15',
 'Editor': 'Marvel Comics',
 'Alineación': 'good'}

In [18]:
heroe_620 = obtener_heroe(620)
df_620 = pd.DataFrame([heroe_620])
df_620

,ID,Nombre,Inteligencia,Fuerza,Velocidad,Durabilidad,Potencia,Combate,Genero,Etnia,Altura,Peso,Color_Ojos,Color_Pelo,Nombre Completo,Alias,Lugar de nacimiento,Primera aparición,Editor,Alineación
0,620,Spider-Man,90,55,67,75,74,85,Male,Human,178 cm,74 kg,Hazel,Brown,Peter Parker,"Spiderman, Bag-Man, Black Marvel, Captain Univ...","New York, New York",Amazing Fantasy #15,Marvel Comics,good
